<div style="margin: 0 auto 30px; height: 60px; border: 2px solid gray; border-radius: 6px;">
  <div style="float: left;"><img src="img/epfl.png" /></div>
  <div style="float: right; margin: 20px 30px 0; font-size: 10pt; font-weight: bold;"><a href="https://moodle.epfl.ch/course/view.php?id=18253">COM202 - Signal Processing</a></div>
</div>
<div style="clear: both; font-size: 30pt; font-weight: bold; color: #483D8B;">
    Lab 9: Audio signal processing
</div>

In this two-part notebook we will learn how to use discrete-time filters to extract specific audio frequencies from an audio file and we will consider some practical issues when implementing filtering algorithms in software.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import IPython
import scipy.signal as sp
from scipy.io import wavfile

In [ ]:
plt.rcParams["figure.figsize"] = (14,4)

# Working with audio using SciPy

The [SciPy library](https://scipy.org/) is a Python library implementing _"fundamental algorithms for scientific computing"_; among the various subpackages included in SciPy, the [signal processing module](https://docs.scipy.org/doc/scipy/reference/signal.html#module-scipy.signal) provides us with a rich set of ready-made routines for reading, writing, playing and filtering audio files. 

In the following subsections please find a brief overview of the main `scipy.signal` tools that we will use later; click on the function name to access the official online documentation.

## Discrete-time filters

In this section we will consider generic discrete-time filters with transfer function

$$
    H(z) = \frac{b_0 + b_1 z^{-1} + \ldots + b_{N-1}z^{-N}}{1 + a_1 z^{-1} + \ldots + a_{N-1}z^{-M}};
$$

most SciPy functions will return or accept as input the transfer function coefficients as two distinct NumPy arrays, which by convention we will call `b` and `a`.

### Filter design

Scipy provides all the fundamental filter design routines; in this notebook we will only use a few of them:
 * [`scipy.signal.butter()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.butter.html#scipy.signal.butter) to design Butterworth filters (monotonic frequency response)
 * [`scipy.signal.ellip()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.ellip.html#scipy.signal.ellip) for elliptical filters (sharp transition bands)
 * [`scipy.signal.remez()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.remez.html#scipy.signal.remez) to design linear-phase optimal FIR filters using the Parks-McClellan algorithm

### Frequency response, poles, zeros

Once you have the $a_k$ and $b_k$ coefficients for the transfer function, you can look at the filter properties using 

 * [`scipy.signal.freqz()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.freqz.html#scipy.signal.freqz)  to plot the frequency response of a filter 
 * [`scipy.signal.tf2zpk()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.tf2zpk.html#scipy.signal.tf2zpk) to compute the poles and the zeros of the transfer function

As an example, the following code block designs a 6th-order lowpass elliptic filter with cutoff frequency $\omega_c = \pi/4,$  10% maximum ripple in passband and 40dB attenuation in stopband. The filter's coefficients are used to plot the magnitude response over the positive frequencies and show the locations of the poles and the zeros:

In [ ]:
b, a = sp.ellip(6, .1, 40, 0.25, 'lowpass')

w, H = sp.freqz(b, a);
plt.plot(w, (np.abs(H)));

In [ ]:
# plot unit circle
uc = np.exp(1j * np.linspace(0, 2 * np.pi, 100))
plt.plot(uc.real, uc.imag, 'k')
plt.axis('equal');

# plot poles and zeros
z, p, _ = sp.tf2zpk(b, a)
plt.plot(z.real, z.imag, 'og')
plt.plot(p.real, p.imag, 'xr');

### Filtering data

Finally, to process a data array with a given filter, you can use [`scipy.signal.lfilter()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.lfilter.html#scipy.signal.lfilter). For example, here are the first 100 samples of the impulse response of a leaky integrator:

In [ ]:
lam = 0.95
plt.stem(sp.lfilter([1-lam], [1, -lam], np.r_[1, np.zeros(99)]));

### Exercise

Plot the first 100 samples of the impulse response of the elliptic filter designed in the previous subsection

In [ ]:
# your code here

## Handling audio files

### Reading and writing WAV files

SciPy's [input-output module](https://docs.scipy.org/doc/scipy/reference/io.html) provides us with convenient functions to read and write audio files in the uncompressed [WAV format](https://en.wikipedia.org/wiki/WAV):

 * [`scipy.io.wavfile.read()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.wavfile.read.html#scipy.io.wavfile.read) to load an audio file
 * [`scipy.io.wavfile.write()`](https://docs.scipy.org/doc/scipy/reference/io.html#module-scipy.io.wavfile) to write audio data to disk in WAV format
 
`wavfile.read()` returns the audio data as a NumPy array whose elements can be in a variety of numerical formats, although most often they will be 16-bit signed integers. For convenience, since we're not concerned with memory constraints in this notebook, we will convert audio data to floating point values between -1 and 1. 

Note also that if the audio file is stereo, the data returned by `wavfile.read()` will be a two-dimensional array. Just to keep things simple, we will convert stereo data to mono by taking the average of the two channels.

### Playing audio

Finally, the [IPython package](https://ipython.org/), written by the same authors of the Jupyter framework, is a Python framework geared at interactive and multimedia-based computing. In this notebook we are interested playing audio files and to this aim we will use the function `IPython.display.Audio()` which returns an interactive widget.

In [ ]:
Fs, s = wavfile.read('data/sm.wav')
IPython.display.Audio(s, rate=Fs)

## Putting it all together in a class

In the rest of this notebook it will be useful to collect many audio manipulation routines in a Python class. Let's start with a simple implementation that handles reading a file and converting the data to our preferred format (mono, floating point):

In [ ]:
class AudioData:
    def __init__(self, filename: str):
        self.rate, self.data = wavfile.read(filename)
        channels = self.data.shape[1] if len(self.data.shape) > 1 else 1
        duration = int(self.data.size / channels / self.rate)
        print(f'duration {duration} seconds, sampling rate {self.rate} Hz')
        # if necessary, convert data to floating point and normalize 
        if self.data.dtype == np.int16:
            self.data = self.data / 32768.0
        elif self.data.dtype != np.float32:
            raise 'unsupported WAV format'
        # convert to mono if multichannel
        if len(self.data.shape) > 1:
            print('converting stereo data to mono')
            self.data = np.sum(self.data, axis=1) / self.data.shape[1]

### Dynamically adding methods

Since we will be adding functionalities to our base class, instead of rewriting the entire class definition every time we will be adding new methods _dynamically_. Python allow us to do so via the `setattr` function.

For instance, here is how we can add a `play` method to play the audio data:

In [ ]:
def play(self) -> object:
    return IPython.display.Audio(self.data, rate=self.rate)

setattr(AudioData, 'play', play)

Now we can use the class like so:

In [ ]:
audio = AudioData('data/sm.wav')
audio.play()

### Exercise: add a method to filter and play

Add a `play_filtered` method to the `AudioData` class that takes the `b` and `a` coefficients of a transfer function and returns a widget that plays the filtered audio data.

In [ ]:
def play_filtered(self, b: np.ndarray, a: np.ndarray) -> object:
    ... # your code here

setattr(AudioData, 'play_filtered', play_filtered)

Let's see what happens if we filter the data with a leaky integrator:

In [ ]:
lam = 0.99
audio.play_filtered([1-lam], [1, -lam])

# Looking at the audio data

When faced with a new dataset, the first thing we want to do is look at the data itself. By now we know we will want plots both in the time and in the frequency domain. Since we are dealing with audio data, it's probably convenient to introduce a small utility function that we can use to extract a portion of the audio given a start and stop time in seconds:

In [ ]:
def audio_slice(self, start:float, stop:float) -> [np.ndarray, np.ndarray]:
    start = 0 if start is None else int(start * self.rate)
    stop = len(self.data) if stop is None else int(stop * self.rate)
    return np.arange(start, stop) / self.rate, self.data[start:stop]

setattr(AudioData, 'audio_slice', audio_slice)

## Time domain

Let's add a method to our AudioData class to plot a user-defined portion of the time-domain waveform:

In [ ]:
def plot(self, start=None, stop=None):
    plt.plot(*self.audio_slice(start, stop))
    plt.xlabel("time [s]")
    
setattr(AudioData, 'plot', plot)    

In [ ]:
audio.plot()

Focusing on a smaller time range shows us (if present) the driving rhythmic pulse of a piece: 

In [ ]:
audio.plot(11.5, 12.5)

## Frequency domain

The next step is being able to plot the DFT of any given portion of the signal.

### Exercise: plotting the magnitude spectrum

Use your previous work in Lab 4 to write a function that plots the magnitude spectrum of a portion of the signal; plot only the positive frequencies and label the frequency axis in hertz.

In [ ]:
def plot_mag_spec(self, start=None, stop=None):
    ... # your code here
    
setattr(AudioData, 'plot_mag_spec', plot_mag_spec)   

In [ ]:
audio.plot_mag_spec(10,12)

## Time-frequency representation

In music, as we mentioned, the rhythmic part is apparent in the time domain as very localized energy bursts, usually associated to drum hits. The harmonic content is best viewed in the frequency domain, obviously. So sometimes you want both at the same time.

Fortunately, SciPy provides us with a ready-made spectrogram routine, which we can use with just a few tweaks:

In [ ]:
def plot_spectrogram(self, start=None, stop=None):
    f, t, spg = sp.spectrogram(self.audio_slice(start, stop)[1], self.rate)
    spg[spg == 0] = np.min(spg[spg > 0])
    plt.pcolormesh(t, f, 10 * np.log10(spg))
    plt.ylabel('frequency [Hz]');
    plt.xlabel('time [s]');
    
setattr(AudioData, 'plot_spectrogram', plot_spectrogram)   

In [ ]:
audio.plot_spectrogram()

## Signal level

In Lab 7 we have seen how to implement a VU-meter using a Leaky integrator. Let's add this functionality to our AudioData class.

### Exercise: plotting the signal level

Add a function `vu_meter` to the AudioData class that returns the estimated [RMS level](https://www.physik.uzh.ch/local/teaching/SPI301/LV-2015-Help/lvanlsconcepts.chm/What_Is_RMS_Level.html#:~:text=The%20RMS%20level%20of%20a,as%20noise%20or%20periodic%20signals.) of the signal over a given time span. Use a moving average filter (like in lab 7) to compute the local power. Also return the time indexes to facilitate plotting

In [ ]:
def vu_meter(self, start=None, stop=None, span_ms=100):
    t, x = self.audio_slice(start, stop)
    M = ...  # compute the length of the MA filter from span_ms
    rms = ...  # your code here
    return t, rms
    
setattr(AudioData, 'vu_meter', vu_meter)   

In [ ]:
audio.plot(0, 4)
plt.plot(*audio.vu_meter(0, 4));

# Audio filters

In this section we will use different filters to extract different frequency regions of an audio file. Let's load one of the files available with this notebook (but you can of course use your own audio file!)

In [ ]:
audio = AudioData('data/sm.wav')
#audio = AudioData('data/groove.wav')

audio.play()

## Extracting the low frequencies

Suppose we are interested in hearing more clearly the bass part; since a bass guitar plays notes with pitch from about 40 to 400 Hz, we could try to filter the audio with a lowpass with cutoff 200Hz to focus on the range of the instrument.

### IIR filtering

Let's start with an elliptic filter for which we can use the `sp.ellip` design routine. Let's first convert the desired cutoff frequency in Hertz to a normalized value that the function understands; according to [the specs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.ellip.html#scipy.signal.ellip), the cutoff frequency should be normalized so that a value of 1 corresponds to half the sampling frequency of the signal. 

We are using a 8-th order filter with 60dB attenuation in stopband and 10% max ripple. Feel free to play with these design parameters and see the effects that they have on the output.

In [ ]:
fc = 200  # cutoff frequency in Hz
wc = fc / (audio.rate / 2)
b_low, a_low = sp.ellip(8, .1, 60, wc)

audio.play_filtered(b_low, a_low)

### FIR filtering

We can try to extract the bass using a linear-phase FIR but, in order to match the performance of the elliptic IIR, the filter will be very long; indeed, since we are selecting a very small passband and a very high stopband attenuation, we will need to use on the order of 1000 taps.

To design the filter we can use the [built-in frequency normalization](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.remez.html#scipy.signal.remez) provided by SciPy's `remez()` function; we choose an odd number of taps in order to obtain a type I filter (symmetric with integer delay).

In [ ]:
fc = 200  # cutoff frequency in Hz
tb = 50  # width of transition band in Hz
M = 1001  # length of the filter
h_low = sp.remez(
    M, 
    [0, fc-tb/2, fc+tb/2, audio.rate/2], 
    [1, 0], 
    [1, 1], 
    fs=audio.rate, 
    maxiter=50,
)

audio.play_filtered(h_low, [1])

### IIR vs FIR

Let's compare the performance of the two filters by looking at their magnitude response. We will focus on the low-frequency part of the spectrum

In [ ]:
PTS = 2048  # by default freqz uses 512 points, we want more
w, H1 = sp.freqz(b_low, a_low, PTS);
w, H2 = sp.freqz(h_low, [1], PTS);

f = np.linspace(0, audio.rate / 2, PTS)  # axis label in Hz

f_max = 800  # let's look at the spectrum up to 800 Hz
ix = int(PTS * f_max / (audio.rate / 2))
plt.plot(f[:ix], 20 * np.log10(np.abs(H1[:ix])), label='elliptic IIR');
plt.plot(f[:ix], 20 * np.log10(np.abs(H2[:ix])), label='optimal FIR');
plt.xlabel("f [Hz]")
plt.ylabel('dB');
plt.legend();

An FIR filter matching the 8th-order elliptic lowpass turns out to be very expensive computationally, with over one thousand operations per output sample. Such a long impulse response also introduces a processing delay of $(M-1)/2$ samples, which could be problematic in real-time applications. So, is an FIR worth the effort? 

To see why it might be so, consider the problem of _removing_ the bass from the original signal. The intuitive approach is to simply subtract the bass part extracted by the filter. Let's try this using the IIR:

In [ ]:
IPython.display.Audio(audio.data - sp.lfilter(b_low, a_low, audio.data), rate=audio.rate)

The reason why this doesn't work is because the processing delay introduced by the filter causes the input and the output to be out of alignment.

With a linear-phase FIR, on the other hand, we know the exact value of the processing delay, $(M-1)/2$ samples, and so we can re-align input and output by delaying the input by $(M-1)/2$:

In [ ]:
delay = int((M-1) / 2)
diff = np.r_[np.zeros(delay), audio.data[:-delay]] - sp.lfilter(h_low, [1], audio.data)

IPython.display.Audio(diff, rate=audio.rate)

A big win for linear phase!

## Extracting the treble

Just as we tried to extract the bass, we can try to extract parts of the drum pattern. Usually, we can get a good feel for the hi-hat and cymbals by keeping the frequencies above 7KHz and discarding the rest

### Exercise: extracting the treble with an FIR

Design an FIR highpass with cutoff 4 kHz; once again, choose an odd number of taps in order to obtain a type I filter (symmetric with integer delay). Experiment with the length of the filter until you obtain an attenuation of at least 50 dB in the stopband.

In [ ]:
fc = ...  # cutoff frequency in Hz
tb = ...  # width of transition band in Hz
M = ...   # length of the filter
h_high = sp.remez(...) 

# plot the magnitude response in dB to check attenuation in stopband
w, H3 = sp.freqz(h_high, [1], PTS);
plt.plot(...);
plt.ylabel('dB');

In [ ]:
audio.play_filtered(h_high, [1])

## Now do it all again!

Go back to the beginning of this section and run all the cells after selecting a different audio file. Feel free to use your own (but convert it to WAV format first, if it's in a compressed format such as MP3)